In [1]:
import hwacctools.quantization.quant as q
from stim_lib.stimulus_gen import generate_qracc_inputs
import numpy as np
import plotly.express as px
import pandas as pd
import seaborn as sns

sns.set_theme()
df = pd.DataFrame

actBits = 5
weightBits = 8
outBits = 8
nActs = 10
nWeights = 3
nOuts = nActs - nWeights + 1

a = quantized_tensor((nActs,nActs),actBits) 
w = quantized_tensor((nWeights,nWeights),weightBits)
o = scaling_quantized_convolution(a,w,outBits,internalPrecision=8)

o.real_values - convolve_fake_quantized(a,w)
# np.round(o.real_values,3) - np.round(convolve_reals(a,w),3)
# np.allclose(o.real_values,convolve_reals(a,w),rtol=0.001)
# Limit numpy print precision
np.set_printoptions(precision=4)
o.real_values, convolve_fake_quantized(a,w)


NameError: name 'quantized_tensor' is not defined

In [ ]:
w,x,wx_outBits = generate_qracc_inputs(
    wDimX = 32, #nColumns
    wDimY = 128, #nRows
    xBatches = 10,
    xTrits = 1,
    outBits = 4,
    seed = 0,
    weight_mode = 'binary',
    col_symmetric=False,
    rangeBits = 6
)
wx = (w @ x.T).T[::-1].T

rtl = np.loadtxt('../tb/qracc/inputs/adc_out_rtlsims.txt')
px.scatter(x=rtl.flatten(),y=wx.flatten())


[STIM_GEN] Generating random weights


In [15]:
df(wx)

,0,1,2,3,4,5,6,7,8,9
0,-2,-4,-16,-1,-5,4,6,0,4,3
1,-1,3,-16,6,-5,5,0,0,4,5
2,-3,6,-8,-3,-6,18,3,-1,0,-3
3,-11,5,-3,-4,-5,6,6,-6,7,2
4,-6,6,-7,0,-9,8,3,-5,3,-2
5,-4,7,-14,8,1,13,4,-1,9,9
6,-8,8,-8,6,-1,9,-4,-7,4,4
7,0,12,-3,-4,-4,5,3,-5,3,0
8,-7,2,-9,1,-2,15,5,-13,2,-3
9,-1,2,-20,3,-3,11,1,-9,-4,-1


In [13]:
df(wx_outBits)

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,0,1,-1,0,-1,2,1,0,-1,-1,...,1,1,-1,0,0,0,0,2,-1,3
1,1,1,0,1,0,2,1,0,0,-1,...,0,0,2,0,1,0,1,2,3,1
2,0,0,-1,-2,-2,-1,-2,-2,-4,-3,...,-4,-2,-1,-1,-2,-3,-3,-3,-5,-2
3,1,0,0,1,0,1,-1,0,1,0,...,1,1,2,-1,1,1,2,-1,0,0
4,1,1,4,1,2,3,2,1,3,2,...,2,2,4,2,2,0,2,0,1,2
5,-2,-2,-2,-2,-3,0,-1,-1,-1,-1,...,-1,0,-2,0,0,2,-3,-1,-1,-1
6,-1,1,-1,-1,0,2,1,-1,0,0,...,-1,0,0,0,-3,1,0,1,-2,0
7,-4,-4,-2,-1,-2,-4,-2,-1,-3,-5,...,-3,-2,-3,-3,-3,-4,-2,-3,-3,-1
8,-1,0,1,1,1,1,2,3,0,0,...,3,3,0,2,1,0,2,2,2,1
9,-1,-1,-1,-3,-2,-1,-2,0,-2,-1,...,-1,-2,0,-1,-2,-2,-2,-3,-2,-3
